In [1]:
import websocket
import thread
import json
import MySQLdb
import pandas as pd

from pprint import pprint
from time import time

In [4]:
class GDAXStore(object):
    def __init__(self):
        self.host= "gdax.ch7tzkxzans9.us-east-1.rds.amazonaws.com"
        self.user= "divir94"
        self.password= "eternity16!"
        self.db= "gdax"
        self.gdax_feed = 'wss://ws-feed.gdax.com'
        
        self.commit_frequency = 5 # in secs
        self.close_time = 20
        self.debug = False
        self.num_rows = 0
        
    def get_mysql_conn(self):
        conn = MySQLdb.connect(host=self.host,
                               user=self.user,
                               passwd=self.password,
                               db=self.db)
        return conn
    
    def get_websocket(self):
        websocket.enableTrace(True)
        ws = websocket.WebSocketApp(self.gdax_feed,
                                    on_open=self.on_open,
                                    on_message=self.on_message,
                                    on_error=self.on_error,
                                    on_close=self.on_close)
        return ws

    def on_message(self, ws, message):
        resp = json.loads(message)
        if self.debug:
            print 'Message arrived'
        self.execute_sql(resp)
        
        time_since_last_commit = time() - self.last_commit_time
        time_since_start = time() - self.start_time

        if time_since_last_commit > self.commit_frequency:
            self.commit()
            
        if self.close_time and time_since_start > self.close_time:
            ws.close()

    def on_error(self, ws, error):
        print 'Message error:\n{}'.format(error)

    def on_close(self, ws):
        self.conn.close()
        print '### Closing db connection and websocket ###'

    def on_open(self, ws):
        def run(*args):
            print 'Running websocket'
            subscribe_request = {
                'type': 'subscribe',
                'product_ids': ['BTC-USD']
            }
            ws.send(json.dumps(subscribe_request))
            print 'Thread terminating...'

        thread.start_new_thread(run, ())


    def execute_sql(self, message):
        columns = ', '.join(message.keys())
        values = ', '.join(map(lambda x: '"{}"'.format(x), message.values()))
        sql = """INSERT INTO gdax.OrderBookUpdates ({}) VALUES ({})""".format(columns, values)
        if self.debug:
            print sql

        try:
            self.cursor.execute(sql)
            self.num_rows += 1
        except Exception as e:
            print 'Rolling back:\n{}'.format(e)
            print self.conn.rollback()
    
    def commit(self):
        self.conn.commit()
        print 'committed {} rows'.format(self.num_rows)
        self.num_rows = 0
        self.last_commit_time = time()

    def get_db(self):
        df = pd.read_sql('select top 100 * from gdax.OrderBookUpdates;', con=self.conn)    
        return df

    def run(self):
        self.start_time = time()
        self.last_commit_time = self.start_time

        self.conn = self.get_mysql_conn()
        self.cursor = self.conn.cursor()
        
        ws = self.get_websocket()
        ws.run_forever()

store = GDAXStore()
store.run()

--- request header ---
GET / HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: ws-feed.gdax.com
Origin: http://ws-feed.gdax.com
Sec-WebSocket-Key: MxTI+AYNHBaZu7/ZLpfBOQ==
Sec-WebSocket-Version: 13
Cookie: __cfduid=dba5bc1e7164786a30b121e4758daad6b1497847046


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Mon, 19 Jun 2017 05:20:16 GMT
Connection: upgrade
Upgrade: websocket
Sec-WebSocket-Accept: Q0kdTs9F+1TuHGU0Vc5T9FBHdkY=
Sec-WebSocket-Version: 13
WebSocket-Server: uWebSockets
Strict-Transport-Security: max-age=15552000; includeSubDomains; preload
X-Content-Type-Options: nosniff
Server: cloudflare-nginx
CF-RAY: 371409c99b87473a-EWR
-----------------------
send: '\x81\xb1\x95\xd7\xac9\xee\xf5\xd8@\xe5\xb2\x8e\x03\xb5\xf5\xdfL\xf7\xa4\xcfK\xfc\xb5\xc9\x1b\xb9\xf7\x8eI\xe7\xb8\xc8L\xf6\xa3\xf3P\xf1\xa4\x8e\x03\xb5\x8c\x8e{\xc1\x94\x81l\xc6\x93\x8ed\xe8'


Running websocket
Thread terminating...
committed 220 rows
committed 239 rows
committed 250 rows


send: '\x88\x82s~7Cp\x96'


### Closing db connection and websocket ###


In [83]:
df = pd.read_csv('test.csv', index_col=0)
df.head()

,client_oid,funds,maker_order_id,order_id,order_type,price,product_id,reason,remaining_size,sequence,side,size,taker_order_id,time,trade_id,type
0,NaN,NaN,NaN,25f5dda8-1185-4f37-9b9b-83fc8d1fed71,NaN,2520.57,BTC-USD,canceled,0.1000,3361541607,sell,NaN,NaN,2017-06-18 19:21:09.815,NaN,done
1,NaN,NaN,NaN,fea41400-c952-4cb8-a6ee-16fbbcf9f782,NaN,2517.02,BTC-USD,canceled,0.2000,3361541608,buy,NaN,NaN,2017-06-18 19:21:09.820,NaN,done
2,NaN,NaN,NaN,66d172d9-2b15-4428-a789-49a27bf405cb,NaN,2520.68,BTC-USD,canceled,0.3457,3361541609,sell,NaN,NaN,2017-06-18 19:21:09.829,NaN,done
3,NaN,NaN,NaN,fcf346ea-4412-4ae2-8369-20c41c258f14,NaN,2517.02,BTC-USD,canceled,0.1000,3361541610,buy,NaN,NaN,2017-06-18 19:21:09.836,NaN,done
4,71225c3d-3fc4-4146-b70e-c398fa1e679a,NaN,NaN,64c408d4-54ce-49ed-949a-ad3825703761,limit,2517.01,BTC-USD,NaN,NaN,3361541611,buy,0.1,NaN,2017-06-18 19:21:09.841,NaN,received


In [18]:
# Import smtplib for the actual sending function
import smtplib

address = 'divir94@gmail.com'

message = """\
From: {}
To: {}
Subject: "Hello"
""".format(address, address)

# Send the message via our own SMTP server, but don't include the
# envelope header.
s = smtplib.SMTP('localhost', 1025)
s.sendmail(address, [address], message)
s.quit()

(221, 'Bye')

In [17]:
import smtplib
import email.utils
from email.mime.text import MIMEText

# Create the message
msg = MIMEText('This is the body of the message.')
msg['To'] = email.utils.formataddr(('Recipient', address))
msg['From'] = email.utils.formataddr(('Author', address))
msg['Subject'] = 'Simple test message'

server = smtplib.SMTP('localhost', 1025)
server.set_debuglevel(True) # show communication with the server
try:
    server.sendmail(address, [address], msg.as_string())
finally:
    server.quit()

send: 'ehlo macbook-pro-2.home\r\n'
reply: '502 Error: command "EHLO" not implemented\r\n'
reply: retcode (502); Msg: Error: command "EHLO" not implemented
send: 'helo macbook-pro-2.home\r\n'
reply: '250 macbook-pro-2.home\r\n'
reply: retcode (250); Msg: macbook-pro-2.home
send: 'mail FROM:<divir94@gmail.com>\r\n'
reply: '250 Ok\r\n'
reply: retcode (250); Msg: Ok
send: 'rcpt TO:<divir94@gmail.com>\r\n'
reply: '250 Ok\r\n'
reply: retcode (250); Msg: Ok
send: 'data\r\n'
reply: '354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: End data with <CR><LF>.<CR><LF>
data: (354, 'End data with <CR><LF>.<CR><LF>')
send: 'Content-Type: text/plain; charset="us-ascii"\r\nMIME-Version: 1.0\r\nContent-Transfer-Encoding: 7bit\r\nTo: Recipient <divir94@gmail.com>\r\nFrom: Author <divir94@gmail.com>\r\nSubject: Simple test message\r\n\r\nThis is the body of the message.\r\n.\r\n'
reply: '250 Ok\r\n'
reply: retcode (250); Msg: Ok
data: (250, 'Ok')
send: 'quit\r\n'
reply: '221 Bye\r\n'
rep